In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Test Notebook for Training on the FileTreeSet

In [ ]:
import datajoint as dj

import os
import torch
import numpy as np
import pickle 
from allenobs.schemas import datasets, allen

import nnfabrik
from nnfabrik import main, builder
import shutil
# import nnvision
from numpy import linalg
import matplotlib.pyplot as plt
import seaborn as sns
from neuralpredictors.data.datasets import FileTreeDataset
# import nnvision
import random
from glob import glob
from collections import OrderedDict
from itertools import zip_longest

# Get Dataloader

In [ ]:
key = datasets.Dataset().fetch("KEY")[0]

In [ ]:
datasets.Dataset() & 'exporter_class != "allenobs.exporters.natural_scenes.NaturalScenes"'

In [ ]:
(datasets.Dataset() & key).fetch('dataset')

In [ ]:
datasets.AgglomerateDataset() * datasets.AgglomerateExporter()

In [ ]:
datasets.AgglomerateDataset().save_to_folder(key={'exporter_hash': '028f7ad795ce5561f974b7efc9390ac3'})

In [ ]:
# key = (datasets.AgglomerateDataset() * datasets.AgglomerateExporter() & dict(exporter_hash='7239d9b307aea198039f1bf6733dbcd8')).fetch("KEY", limit=1)[0]
# datasets.AgglomerateDataset().save_to_folder(key, zip=False, base_path='./allen')

In [ ]:
# datasets.Dataset().save_to_folder(key, zip=False)

In [ ]:
mouse_allen_scene_loaders(**dict(
    paths=['container511498742-experiment_id511498742-NaturalScenes-248296732335b73512bdd95b246a8c95.pkl'],
    batch_size=32,
    normalize=True,
    file_tree=True,
    areas=('VISp',)
))

In [3]:
# c/data/ge path 
paths = ['data/allen-agglomerate-NaturalScenesOracle-028f7ad795.zip']


In [5]:
dataset_fn = 'nnsysident.datasets.mouse_loaders.mouse_allen_scene_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=32,
    normalize=True,
    file_tree=True,
    areas=('VISp',)
)
dataloaders = builder.get_data(dataset_fn, dataset_config)

Unzipping data/allen-agglomerate-NaturalScenesOracle-028f7ad795.zip into /notebooks/data


# Get NNkonsti Model

In [ ]:
# model_fn = 'nnvision.models.se_core_spatialXfeature_readout'
model_fn = 'nnvision.models.se_core_gauss_readout'

model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':2,
   'input_kern': 15,
   'gamma_input': 10, #6.3831,          
   'gamma_readout': 0.0076,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels':  16,
   'n_se_blocks': 0,
   'depth_separable': True,
#    'normalize': True
   'init_mu_range': 0.001,
   'init_sigma_range': 0.4
}
model = builder.get_model(model_fn=model_fn, 
                          model_config=model_config, 
                          dataloaders=dataloaders, 
                          seed=1000)
model

# Get Trainer

In [ ]:
trainer_fn = 'nnvision.training.nnvision_trainer'
trainer_config = dict(max_iter=100,
                      verbose=False, 
                      lr_decay_steps=2,
                      avg_loss=False, 
                      patience=5,
                      lr_init=.0041)
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [ ]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1000)

In [ ]:
model

In [ ]:
x = torch.Tensor(1, 1, 49, 62).fill_(0).to(0)
x.requires_grad=True
y = model(x, '-agglomerate-NaturalScenesOracle-7239d9b307')
print(y.shape)
y[0,200].backward()
plt.imshow(x.grad.cpu().detach().numpy().squeeze(), cmap='gray')